In [1]:
from toy_reg import *
import ot
import os
import warnings
warnings.filterwarnings('ignore')

## pretrained AE

In [2]:
X_train_pre, y_train_pre, regions_train = generate_imbalanced_data(5000,5000, seed = 1)

In [ ]:
XY_tensor =  combine_XY(X_train_pre, y_train_pre)

set_seed(111)
# Create a DataLoader
# Initialize and train the autoencoder
autoencoder = Autoencoder(input_dim=6, latent_dim=3, hidden_dim=512)
autoencoder = train_autoencoder(autoencoder, XY_tensor, num_epochs=1000, learning_rate=1e-4, device='cuda')
autoencoder.eval()

## CoDSA with pretrained AE

In [ ]:
list_result = []
list_origin = []
W_list = [[] for j in range(10)]
ce_tf=[]


num_sim=10
num_split= len(ratio_list )
best_indices = np.empty((num_sim, num_split, 2), dtype=int)
# best_test_errors[i,j] will store the corresponding test error.
best_test_errors = np.empty((num_sim, num_split))



# Set data sizes.
n_minority = 800
n_majority = 3200
n_val = 400
n_test = 800
n_train =4000
m=2*max(candidate_m_syn)

#hyperparameter sets
ratio_list = [x/10 for x in list(range(1,11))] #split ratio r
candidate_m_syn=  [int(n_train*round(x * 0.1, 2)) for x in range(21)] #synthetic sample size m/n
candidate_alpha_scale= [0.1, 0.2, 0.3,0.4, 0.5,0.6, 0.7,0.8, 0.9] #alpha

X_total, y_total, regions_total = generate_imbalanced_data(
        n_minority + n_val + n_test, 
        n_majority + n_val + n_test, 
        seed=0
    )

if not os.path.exists("synthetic_tf"):
    os.makedirs("synthetic_tf")

for j in range(10):
    set_seed(j)
    
    # Separate the minority and majority groups based on region labels.
    # Here, we assume that region label 0 indicates minority and 1 indicates majority.
    minority_idx = np.where(regions_total == 0)[0]
    majority_idx = np.where(regions_total == 1)[0]

    # Shuffle indices separately
    np.random.shuffle(minority_idx)
    np.random.shuffle(majority_idx)

    # For minority:
    min_train_idx = minority_idx[:n_minority]
    min_val_idx   = minority_idx[n_minority : n_minority + n_val]
    min_test_idx  = minority_idx[n_minority + n_val : n_minority + n_val + n_test]

    # For majority:
    maj_train_idx = majority_idx[:n_majority]
    maj_val_idx   = majority_idx[n_majority : n_majority + n_val]
    maj_test_idx  = majority_idx[n_majority + n_val : n_majority + n_val + n_test]

    # Combine directly for each set
    X_train_orig = np.vstack((X_total[min_train_idx], X_total[maj_train_idx]))
    y_train_orig = np.vstack((y_total[min_train_idx], y_total[maj_train_idx]))
    regions_train = np.concatenate((regions_total[min_train_idx], regions_total[maj_train_idx]))

    X_val = np.vstack((X_total[min_val_idx], X_total[maj_val_idx]))
    y_val = np.vstack((y_total[min_val_idx], y_total[maj_val_idx]))
    regions_val = np.concatenate((regions_total[min_val_idx], regions_total[maj_val_idx]))

    X_test = np.vstack((X_total[min_test_idx], X_total[maj_test_idx]))
    y_test = np.vstack((y_total[min_test_idx], y_total[maj_test_idx]))
    regions_test = np.concatenate((regions_total[min_test_idx], regions_total[maj_test_idx]))


    XY_train_orig = combine_XY(X_train_orig, y_train_orig)
    
    n_train= XY_train_orig.shape[0]

    origin_model, mse_val_origin, mse_test_origin, y_val_opred, y_test_opred = train_and_evaluate(
    X_train_orig, y_train_orig, X_val, y_val, X_test, y_test)
    list_origin.append([mse_val_origin, mse_test_origin])
    print([mse_val_origin, mse_test_origin])

    all_result=[]
    for split_ratio in ratio_list:

        n_diff1 = int(split_ratio * n_minority)
        n_diff2 = int(split_ratio * n_majority)

        # Shuffle indices for each group.
        indices_min = np.arange(n_minority)
        np.random.shuffle(indices_min)
        indices_maj = np.arange(n_majority)
        np.random.shuffle(indices_maj)

        # Extract diffusion subset using the shuffled indices.
        XY_diff = np.vstack((
            XY_train_orig[indices_min[:n_diff1], :],
            XY_train_orig[n_minority + indices_maj[:n_diff2], :]
        ))
        regions_diff = np.concatenate((
            regions_train[indices_min[:n_diff1]],
            regions_train[n_minority + indices_maj[:n_diff2]]
        ))
        
        if split_ratio==1:
            XY_reg=None
        else:
            XY_reg = np.vstack((XY_train_orig[n_diff1:n_minority,:],XY_train_orig[(n_minority+n_diff2):,:]))
        

        result=[]
        file_path = f"synthetic_tf/synthetic_X_seed{j}_ratio{split_ratio:.1f}.npy"
        if os.path.exists(file_path):          

            # Build file paths.
            file_X = f"synthetic_tf/synthetic_X_seed{j}_ratio{split_ratio:.1f}.npy"
            file_y = f"synthetic_tf/synthetic_y_seed{j}_ratio{split_ratio:.1f}.npy"

            # Load the synthetic data.
            X_syn_full = np.load(file_X)
            y_syn_full = np.load(file_y)
            
        else:
            with torch.no_grad():        
                U_diff=autoencoder.encoder(torch.tensor(XY_diff, dtype=torch.float32).to('cuda'))

            model = ConditionalDiffusionModel(text_dim=3, cond_dim=1, hidden_dim= 1024, time_embed_dim=128,
                                              num_fc_blocks=10,dropout = 1e-5)
            sampler = DDIMSampler(device=device,noise_steps=1000)
            model = model.to(device)
            trained_model = train_conditional_diffusion(model, U_diff, regions_diff, sampler, num_epochs=2000, batch_size=64, seed=j,device='cuda')


            X_syn_full, y_syn_full = generate(trained_model,autoencoder.decoder,m,0.5,sampler,seed =j)        
            
            np.save(f"synthetic_tf/synthetic_X_seed{j}_ratio{split_ratio:.1f}.npy", X_syn_full)
            np.save(f"synthetic_tf/synthetic_y_seed{j}_ratio{split_ratio:.1f}.npy", y_syn_full)
        

        X_truth,y_truth,_=generate_imbalanced_data(int(m/2), int(m/2),seed=j)
        truth_samples=combine_XY(X_truth,y_truth)
        gen_samples = combine_XY(X_syn_full, y_syn_full)      
        # Compute cost matrix (Euclidean distances)
        M = ot.dist(truth_samples, gen_samples, metric='euclidean')
        a = np.ones((m,)) / m
        b = np.ones((m,)) / m
        W_distance = ot.emd2(a, b, M)
        W_list[j].append(W_distance)
        print("Wasserstein distance:", W_distance)
        
        
        for m_syn in candidate_m_syn:   
            # Generate synthetic data for minority (region 1)
           
            tmp=[]
            if m_syn==0 and split_ratio ==1:
                for alpha_scale in candidate_alpha_scale:
                    tmp.append([np.inf, np.inf])
                result.append(tmp)
                continue
            for alpha_scale in candidate_alpha_scale:
                # Adjust synthetic data: if alpha_scale < 1, use a fraction; if > 1, replicate data accordingly.
                if m_syn ==0:
                    X_train_combined = XY_reg[:,:-1]
                    y_train_combined = XY_reg[:,-1:]    
                else:

                #X_train_combined,y_train_combined = generate(trained_model,m_syn,alpha_scale)
                    X_train_combined = np.vstack((X_syn_full[:int(m_syn*alpha_scale),], X_syn_full[int(m/2):int(m/2+m_syn*(1-alpha_scale)),]))
                    y_train_combined = np.vstack((y_syn_full[:int(m_syn*alpha_scale),], y_syn_full[int(m/2):int(m/2+m_syn*(1-alpha_scale)),]))

                    if split_ratio <1:
                        X_train_combined = np.vstack((X_train_combined,XY_reg[:,:-1]))
                        y_train_combined = np.vstack((y_train_combined,XY_reg[:,-1:]))


                # Train and evaluate on validation set
                _, mse_val, mse_test, _, _ = train_and_evaluate(
                    X_train_combined, y_train_combined, X_val, y_val, X_test, y_test)

                print(f"m_syn={m_syn}, alpha_scale={alpha_scale} -> Validation MSE: {mse_val:.4f} -> Test MSE: {mse_test:.4f}")

                tmp.append([mse_val, mse_test])
            result.append(tmp)

        all_result.append(result)
    
    ax= [all_result[x][:21] for x in range(10)] 
    ax=np.array(ax)
    num_sim, num_split, num_m, num_min_ratio = ax.shape
    val_errors = ax[: , :, :, 0]

    # Find the indices (o, k, l) that minimize the validation error.
    o, k, l = np.unravel_index(np.argmin(val_errors), val_errors.shape)
    best_ratio = ratio_list[o]
    best_m = candidate_m_syn[k]
    best_alpha = candidate_alpha_scale[l]

    
    n_diff1 = int(best_ratio * n_minority)
    n_diff2 = int(best_ratio * n_majority)

    # Shuffle indices for each group.
    indices_min = np.arange(n_minority)
    np.random.shuffle(indices_min)
    indices_maj = np.arange(n_majority)
    np.random.shuffle(indices_maj)

    # Extract diffusion subset using the shuffled indices.
    XY_diff = np.vstack((
        XY_train_orig[indices_min[:n_diff1], :],
        XY_train_orig[n_minority + indices_maj[:n_diff2], :]
    ))
    regions_diff = np.concatenate((
        regions_train[indices_min[:n_diff1]],
        regions_train[n_minority + indices_maj[:n_diff2]]
    ))

    XY_reg = np.vstack((XY_train_orig[n_diff1:n_minority,:],XY_train_orig[(n_minority+n_diff2):,:]))


    

    # Build file paths.
    file_X = f"synthetic_tf/synthetic_X_seed{j}_ratio{best_ratio:.1f}.npy"
    file_y = f"synthetic_tf/synthetic_y_seed{j}_ratio{best_ratio:.1f}.npy"

    # Load the synthetic data.
    X_syn_full = np.load(file_X)
    y_syn_full = np.load(file_y)

    X_train_combined = np.vstack((
        X_syn_full[:int(best_m*best_alpha), :],
        X_syn_full[int(m/2):int(m/2+best_m*(1-best_alpha)), :]
    ))
    y_train_combined = np.vstack((
        y_syn_full[:int(best_m*best_alpha), :],
        y_syn_full[int(m/2):int(m/2+best_m*(1-best_alpha)), :]
    ))

    X_train_combined = np.vstack((X_train_combined, XY_reg[:, :-1]))
    y_train_combined = np.vstack((y_train_combined, XY_reg[:, -1:]))

    ce_one={}
    # Train and evaluate on validation set
    _, mse_val, mse_test, ypred_val, ypred_test = train_and_evaluate(
        X_train_combined, y_train_combined, X_val, y_val, X_test, y_test)
    ce_one['avg']=mse_test
    
    _, mse_val, mse_test, ypred_val, ypred_test = train_and_evaluate(
    X_train_combined, y_train_combined, X_val, y_val, X_test[regions_test==1,], y_test[regions_test==1,])
    ce_one['major']=mse_test
        
        
    _, mse_val, mse_test, ypred_val, ypred_test = train_and_evaluate(
    X_train_combined, y_train_combined, X_val, y_val, X_test[regions_test==0,], y_test[regions_test==0,])
    ce_one['minor']=mse_test       
    
    ce_tf.append(ce_one)
    list_result.append(all_result)


In [14]:
import pickle

with open("result/res_tf.pkl", "wb") as f:
    pickle.dump([list_result,W_list,list_origin,ce_tf], f)


In [7]:
import pickle
import numpy as np

with open("result/res_tf.pkl", "rb") as f:
    list_result,W_list,list_origin,ce_tf,_ = pickle.load(f)

### Summary statistics for Table 2

In [8]:
[np.mean([x['major'] for x in ce_tf]),np.mean([x['minor'] for x in ce_tf]),np.mean([x['avg'] for x in ce_tf])] 

[0.42010826531690004, 0.762041202574482, 0.591074733945691]

In [9]:
[np.std([x['major'] for x in ce_tf]),np.std([x['minor'] for x in ce_tf]),np.std([x['avg'] for x in ce_tf])] 

[0.104574241487262, 0.501055635078655, 0.27182784429640644]